In [1]:
!pip install protobuf transformers cpm_kernels torch>=2.0 gradio mdtex2html sentencepiece accelerate


In [1]:
from transformers import AutoTokenizer, AutoModel, set_seed
from pathlib import Path
# model_dir = 'chatglm2-6b/models--THUDM--chatglm2-6b-int4/snapshots/382cc704867dc2b78368576166799ace0f89d9ef' 
model_dir = "THUDM/chatglm2-6b-int4"
local_model_path = Path("./chatglm2-6b")

tokenizer = AutoTokenizer.from_pretrained(model_dir,
                                          revision = '382cc704867dc2b78368576166799ace0f89d9ef',
                                          cache_dir  = local_model_path,
                                           trust_remote_code=True)
model = AutoModel.from_pretrained(model_dir,
                                   revision = '382cc704867dc2b78368576166799ace0f89d9ef',
                                    cache_dir  = local_model_path,
                                  trust_remote_code=True).half().cuda()
model = model.eval()

Compile parallel cpu kernel gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\xiehu\.cache\huggingface\modules\transformers_modules\THUDM\chatglm2-6b-int4\382cc704867dc2b78368576166799ace0f89d9ef\quantization_kernels_parallel.c -shared -o C:\Users\xiehu\.cache\huggingface\modules\transformers_modules\THUDM\chatglm2-6b-int4\382cc704867dc2b78368576166799ace0f89d9ef\quantization_kernels_parallel.so failed.
Compile cpu kernel gcc -O3 -fPIC -std=c99 C:\Users\xiehu\.cache\huggingface\modules\transformers_modules\THUDM\chatglm2-6b-int4\382cc704867dc2b78368576166799ace0f89d9ef\quantization_kernels.c -shared -o C:\Users\xiehu\.cache\huggingface\modules\transformers_modules\THUDM\chatglm2-6b-int4\382cc704867dc2b78368576166799ace0f89d9ef\quantization_kernels.so failed.


In [2]:
import time
from transformers.generation import GenerationConfig
# model.generation_config = GenerationConfig.from_pretrained("THUDM/chatglm2-6b-int4",cache_dir='chatglm2-6b',trust_remote_code=True)
# model.return_dict_in_generate  = True
# model.output_scores  = True

parameters = {
  "max_length": 512,
  "temperature": 0.1,
  "top_p":0.95,
   "top_k":50,
  "output_scores":True,
  "return_dict_in_generate":True,
  "output_attentions":True,
  "output_hidden_states":True,
}



def chat(prompt, parameters=parameters):
    global model, tokenizer
    
    set_seed(1)
    start = time.time()
    response, _, outputs = model.chat(tokenizer, prompt, history=[],**parameters)
    end = time.time()
    num_tokens = len(tokenizer(response)['input_ids'])
    print('{} tokens took time {:.1f}s, inference speed:{} tokens/s'.format(num_tokens,end-start,num_tokens/(end-start)))
    return response,outputs


In [3]:
prompt = """您好"""
response,outputs = chat(prompt)
print(response)

13 tokens took time 5.4s, inference speed:2.425496067006134 tokens/s
您好！有什么我能帮助您解答的问题吗？


In [4]:
print('input tokens:',len(tokenizer(prompt)['input_ids']))
tokens = tokenizer(response)['input_ids']
print('output tokens:',len(tokens))
for tok in tokens:
    print(tok,tokenizer.decode(tok,skip_special_tokens=False))

input tokens: 4
output tokens: 13
64790 
64792 
30910 
48214 您好
31404 ！
33277 有什么
40601 我能
31934 帮助
55353 您
36295 解答
32184 的问题
55398 吗
31514 ？


In [5]:
print(list(outputs.hidden_states)[0][0][0])
print(list(outputs.hidden_states)[0][0][16])

tensor([[ 0.0077, -0.0167, -0.0165,  ..., -0.0325,  0.0297,  0.0022]],
       device='cuda:0', dtype=torch.float16)
tensor([[-0.0037,  0.0049,  0.0040,  ..., -0.0187, -0.0112, -0.0092]],
       device='cuda:0', dtype=torch.float16)


In [37]:

def build_inputs(tokenizer, query: str,device :str = 'cuda:0'):
    inputs = tokenizer([query], return_tensors="pt")
    inputs = inputs.to(device)
    return inputs

In [9]:
#取出拼接的原始inputs，
input_ids = model.build_inputs(tokenizer,prompt)
print(tokenizer.decode(input_ids['input_ids'][0].tolist()))
for tk in input_ids['input_ids'][0].tolist():
    print(f'{tk}:{tokenizer.decode(tk)}')


[Round 1]

问：您好

答：
64790:
64792:
790:[
30951:R
517:ound
30910:
30939:1
30996:]
13:<0x0A>
13:<0x0A>
54761:问
31211:：
48214:您好
13:<0x0A>
13:<0x0A>
55437:答
31211:：


In [38]:
import time 
##取出encoder的hiden stat

text = 'how are you today?who are you?'
start = time.time()
input_ids = build_inputs(tokenizer,text)
encoder_output = model.transformer(**input_ids)

end = time.time()
print(input_ids)
print('cost {:.3f}'.format(end-start))
print(encoder_output.last_hidden_state)

{'input_ids': tensor([[64790, 64792,   709,   383,   344,  1661, 30987,  9880,   383,   344,
         30987]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'position_ids': tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]], device='cuda:0')}
cost 0.275
tensor([[[-0.4956,  0.2131, -0.8540,  ...,  0.4507, -0.3835, -0.1642]],

        [[-0.4045,  0.0307, -0.9839,  ...,  0.4565, -0.2025, -0.0921]],

        [[-1.5449,  0.8818, -0.3604,  ...,  0.9790, -0.7524,  0.5781]],

        ...,

        [[ 0.0216, -2.9961,  0.9497,  ..., -2.6543,  0.7446,  2.2559]],

        [[ 1.3418, -0.8984, -2.7734,  ..., -1.3682, -1.5859, -2.8555]],

        [[ 2.2266, -0.9355,  2.0469,  ..., -3.0762, -4.4180,  2.8223]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<ToCopyBackward0>)


In [236]:
import torch
import torch.nn.functional as F


def get_layer_h_stats(outputs, layer_index = -1):
    last_layer_h_stats = []
    for token_h_states in list(outputs.hidden_states)[0]:  ## the first hidden stat is encoder output.
        ## token_hstates is the the hidden_states for each token
        ## each hstates is concated with output state of each layer, plus final post norm layer. so the length is num_layers+1 ,eg, chatglm 6b-2 is 29
        token_h_states = list(token_h_states)[layer_index]
        # print(token_h_states.shape)
        last_layer_h_stats.append(token_h_states)

    # Concatenate the tensors along a new dimension
    concatenated = torch.cat(last_layer_h_stats, dim=0)
    # Sum the tensors along the 1st dimension
    mean_pooled = torch.mean(concatenated, dim=0, keepdim=True).view(1,4096)
    normalized_result = F.normalize(mean_pooled, p=2, dim=-1)
    return normalized_result

In [237]:
emb1 = get_layer_h_stats(outputs)

In [242]:
prompt = """please repeat the below sentences:中国的首都是北京"""
response,outputs = chat(prompt)
print(response)

8 tokens took time 0.6s, inference speed:14.338263107201858 tokens/s
中国的首都是北京。


In [243]:
emb2 = get_layer_h_stats(outputs)

In [244]:
similarity = F.cosine_similarity(emb1, emb2, dim=-1)
print(similarity.item()) 

0.64453125


In [233]:
import pandas as pd
import torch.nn as nn
for i in range(len(outputs.scores)):
    probs = nn.functional.softmax(outputs.scores[i], dim=-1)[0]
    print(sorted(probs.tolist(),reverse=True)[:10])

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [45]:
import numpy as np
import torch.nn as nn
token_scores = []
for i,next_token_scores in enumerate(outputs.scores[:-1]):
    token_scores.append(next_token_scores.max().item())
    probs = nn.functional.softmax(next_token_scores, dim=-1)
    print (probs)
print('average logits:{0:.5f}'.format(np.array(token_scores).mean()))

tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cud

In [46]:
prompt = """写一篇500字的描述秋天的小作文"""
response = chat(prompt)
print(response[])

{'max_length': 512, 'num_beams': 1, 'do_sample': True, 'top_p': 0.8, 'temperature': 0.7, 'logits_processor': [<transformers_modules.382cc704867dc2b78368576166799ace0f89d9ef.modeling_chatglm.InvalidScoreLogitsProcessor object at 0x00000218B340EBD0>], 'top_k': 50, 'output_scores': True, 'return_dict_in_generate': True}
376 tokens took time 12.7s, inference speed:29.570891303278632 tokens/s
('秋天是一年中最美丽的季节之一,也是一个充满变化和探索的季节。当树叶开始变成橙色和红色,空气中弥漫着清新的气息时,秋天开始了。在这个美好的季节里,我们可以享受到阳光明媚的天气,感受到大自然的美丽和宁静。\n\n秋天的天气非常好,天空湛蓝,白云在空中自由自在地飘荡。这样的天气总是让人感到心情愉悦,精神焕发。在这个季节里,白天相对比较短,太阳落山的时间也相对较早,所以人们可以有更多的时间去享受阳光,欣赏大自然的美景。\n\n秋天的树叶是这个季节中最美丽的景色之一。树叶开始变成橙色和红色,让整个大地变得充满了活力和热情。在这个季节里,我们可以去公园或者山林里散步,欣赏各种各样的树叶,感受大自然的美丽和宁静。同时,我们也可以看到很多树叶变成了掉落的黄叶,这样的景色也充满了诗意。\n\n秋天也是一个充满探索和发现的季节。在这个季节里,我们可以去探索各种地方,发现不同的美景和自然景观。我们可以去登山,感受高山上的清新空气和美景;我们可以去潜水,探索水下的世界;我们也可以去博物馆,了解历史和文化。\n\n秋天也是一个充满感恩和反思的季节。在这个季节里,我们可以回顾过去,感恩大自然的恩赐和生命的存在;我们也可以珍惜当下的每一刻,感悟生命的意义和价值。同时,我们也可以去帮助需要帮助的人,为社会做出贡献,回应对社会的感恩和回报。\n\n秋天是一个充满变化和探索的季节,也是一个充满感恩和反思的季节。在这个季节里,我们可